# Data Analysis

In [ ]:
import os
import nibabel as nib
import re
import numpy as np
from datetime import datetime
import SimpleITK as sitk
from collections import defaultdict
from scipy.ndimage import label
from tqdm import tqdm

Paths

In [ ]:
image_root = "/home/gkolokolnikov/PhD_project/nf_segmentation_interactive/NFInteractiveSegmentationBenchmarkingPrivate/evaluation/results/images"
mask_root = "/home/gkolokolnikov/PhD_project/nf_segmentation_interactive/NFInteractiveSegmentationBenchmarkingPrivate/evaluation/results/ground_truth"

dataset_names = ["TrainingSet", "TestSet_1", "TestSet_2", "TestSet_3", "TestSet_4"]

Date time extraction

In [ ]:
# Extract datetime from description
def extract_datetime_from_string(meta_str):
    match = re.search(r'\d{1,2}-[A-Za-z]{3}-\d{4} \d{1,2}:\d{1,2}:\d{1,2}(?:\.\d+)?', meta_str)
    if match:
        try:
            return datetime.strptime(match.group(0), "%d-%b-%Y %H:%M:%S.%f")
        except ValueError:
            return datetime.strptime(match.group(0), "%d-%b-%Y %H:%M:%S")
    return None

Analyse the dataset

In [ ]:
def analyze_dataset(dataset_name):
    image_dir = os.path.join(image_root, dataset_name)
    mask_dir = os.path.join(mask_root, dataset_name)

    acquisition_dates = []
    slice_counts = []
    inplane_sizes = []
    volumes = []
    spacings = []
    lesion_counts = []

    file_list = [f for f in os.listdir(image_dir) if f.endswith(".nii.gz")]

    for fname in tqdm(file_list, desc=f"Processing {dataset_name}"):
        try:
            image_path = os.path.join(image_dir, fname)
            mask_path = os.path.join(mask_dir, fname)
            
            img = sitk.ReadImage(image_path)
            mask = sitk.ReadImage(mask_path)

            # Acquisition date from metadata
            desc = ""
            for key in ["descrip"]:
                
                if img.HasMetaDataKey(key):
                    desc = img.GetMetaData(key)
                    break
            dt = extract_datetime_from_string(desc)
            if dt:
                acquisition_dates.append(dt)

            # Shape and spacing
            x, y, z = img.GetSize()
            spacing = img.GetSpacing()
            sp_x, sp_y, sp_z = spacing
            slice_counts.append(z)
            inplane_sizes.append(tuple(sorted((x, y))))  # (smaller, larger)

            # Volume in cm³
            mask_array = sitk.GetArrayFromImage(mask)
            tumor_voxels = np.sum(mask_array > 0)
            voxel_volume_cm3 = np.prod(spacing) / 1000
            volumes.append(tumor_voxels * voxel_volume_cm3)
            spacings.append(tuple(sorted((sp_x, sp_y, sp_z))))

            # Lesion count
            labeled, num_lesions = label(mask_array > 0)
            lesion_counts.append(num_lesions)

        except Exception as e:
            print(f"[{dataset_name}] Error processing {fname}: {e}")

    # Summary
    def iqr_stats(arr):
        median = np.median(arr)
        q1 = np.percentile(arr, 25)
        q3 = np.percentile(arr, 75)
        return median, q3 - q1

    # Compute in-plane split
    small_dims = [s[0] for s in inplane_sizes]
    large_dims = [s[1] for s in inplane_sizes]
    small_sp = [s[0] for s in spacings]
    medium_sp = [s[1] for s in spacings]
    large_sp = [s[2] for s in spacings]

    # Compute stats
    slice_med, slice_iqr = iqr_stats(slice_counts)
    small_med, small_iqr = iqr_stats(small_dims)
    large_med, large_iqr = iqr_stats(large_dims)
    vol_med, vol_iqr = iqr_stats(volumes)
    sp_s_med, sp_s_iqr = iqr_stats(small_sp)
    sp_m_med, sp_m_iqr = iqr_stats(medium_sp)
    sp_l_med, sp_l_iqr = iqr_stats(large_sp)
    lesion_med, lesion_iqr = iqr_stats(lesion_counts)

    # Summary print
    print(f"\n==== {dataset_name} ====")
    print(f"Cases: {len(slice_counts)}")
    if acquisition_dates:
        print(f"Acquisition Date Range: {min(acquisition_dates).date()} to {max(acquisition_dates).date()}")
    else:
        print("Acquisition Date Range: N/A")
    
    min_small = min(s[0] for s in inplane_sizes)
    max_small = max(s[0] for s in inplane_sizes)
    min_large = min(s[1] for s in inplane_sizes)
    max_large = max(s[1] for s in inplane_sizes)
    

    print(f"Slices: {min(slice_counts)} to {max(slice_counts)}; {slice_med:.1f} ± {slice_iqr:.1f}")
    print(f"In-plane size (smaller dim): {min_small} to {max_small}; {small_med:.1f} ± {small_iqr:.1f}")
    print(f"In-plane size (larger dim): {min_large} to {max_large}; {large_med:.1f} ± {large_iqr:.1f}")
    print(f"Spacing: {sp_s_med:.2f} ± {sp_s_iqr:.2f}; {sp_m_med:.2f} ± {sp_m_iqr:.2f}, {sp_l_med:.2f} ± {sp_l_iqr:.2f}")
    print(f"Tumor volume (cm³): {vol_med:.2f} ± {vol_iqr:.2f}")
    print(f"Number of lesions: {lesion_med:.1f} ± {lesion_iqr:.1f}")
    print(f"\n==== ==== ==== ==== ====")

Launch the dataset

In [ ]:
# Run for each dataset
for dataset in dataset_names:
    analyze_dataset(dataset)